Group Members: Sapana Hingrajia, Jake McGraw, Joanna Rios, Himmat Jagdev

## Using ML to predict NBA 2024 Playoffs Results 

In this project, we use techniques learned so far to predict the teams that could make it to the Playoffs. 
We start by using the advanced statistics from 1992 to 2024, then concadenated all the csv gathered.
to build the main df "NBA_TeamStats1992-2024". 
We then apply various machine learning methods and approaches to attempt to create a accurate a model as possible to solve the classification problem at hand.

## Part 0: Data Cleaning and Preparation

Importing NBA_TeamStast Data frame and Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("NBA_TeamStats_1992-2024_df.csv")

df.head()


Renaming columns to become familiar with the advance statistics names

In [ ]:
# Rename the columns needed
df=pd.read_csv("NBA_TeamStats_1992-2024_df.csv")
df_renamed = df.rename(columns={
                                        "Rk": "RANK",
                                        "W": "WINS",
                                        "L": "LOSSES",
                                        "PW": "PYTHAGOREAN WINS",
                                        "PL": "PYTHAGOREAN LOSSES",
                                        "MOV": "MARGIN OF VICTORY",
                                        "SOS": "STRENGTH OF SCHEDULE",
                                        "SRS": "SIMPLE RATING SYSTEM",
                                        "ORtg": "OFFENSIVE RATING",
                                        "DRtg": "DEFENSIVE RATING",
                                        "NRtg": "NET RATING",
                                        "Pace": "PACE FACTOR",
                                        "FTr": "FREE THROW ATTEMPT RATE",
                                        "3PAr": "3-POINT ATTEMPT RATE",
                                        "TS%": "TRUE SHOOTING PERCENTAGE",
                                        "eFG%": "EFFECTIVE FIELD GOAL PERCENTAGE",
                                        "TOV%": "TURNOVER PERCENTAGE",
                                        "ORB%": "OFFENSIVE REBOUND PERCENTAGE",
                                        "FT/FGA": "FREE THROWS PER FIELD GOAL ATTEMPT",
                                        "eFG%.1": "OPPONENT EFFECTIVE FIELD GOAL PERCENTAGE",
                                        "TOV%.1": "OPPONENT TURNOVER PERCENTAGE",
                                        "DRB%": "DEFENSIVE REBOUND PERCENTAGE",
                                        "FT/FGA.1": "OPPONENT FREE THROWS PER FIELD GOAL ATTEMPT",
                                         })


df_renamed.head()

In [ ]:
# Check for missing values
print(df_renamed.isnull().sum())

In [ ]:
# Summary statistics
print(df_renamed.describe())

## Part 1: Exploring the data. 

Determining if same amount of information/data is available accross the years.

In [ ]:
df_renamed

In [ ]:
#Creating a dataframe to evaluate how many rows of data are in each year
df_year = df_renamed[["Year"]]
df_year.plot.hist(color="blue",rwidth=0.8) 


Filtering the df only to visualize teams that made it to the Championship over the years

In [ ]:
#Filtering the df by Championship = 1
df_won_champ = df_renamed.loc[(df_renamed["Championship"].str.contains("1"))]

df_won_champ.head()

In [ ]:
#Creating a df to be used as a table correlation for further analysis

#New df filtered by who won the Championship 
df_renamed.copy()

df_won_champ_corr = df_renamed.drop(columns=["Team","Playoffs"])

df_won_champ_corr = df_won_champ_corr.loc[(df_renamed["Championship"].str.contains("1"))]

#df_won_champ_corr['Championship'].astype(int)
#df_won_champ_corr['Playoffs'].astype(int)

#Correlation table
df_won_champ_corr.head()

In [ ]:
#Correlation table
df_won_champ_corr.corr()

Heat map based on correlation table to identify potential indication of most important variables before starting the prediction

In [ ]:
#correlation table to heatmap using sns
sns.heatmap(df_won_champ_corr.corr())

In [ ]:
#df grouped by Championship
by_win = df_won_champ.groupby('Championship')
by_win["Team"].value_counts()

#Team counts by wins df
team_counts = by_win["Team"].value_counts()

team_counts_by_win = team_counts.reset_index()
team_counts_by_win.columns = ["Championship",'Team', 'Count']

team_counts_by_win


In [ ]:
#Championship column dropped
by_team = team_counts_by_win.drop(columns=['Championship'])

by_team

In order to make a prediction is important to identify which teams have performed better over the years

In [ ]:
#Plotting Team vs age 
by_team.plot(x='Team', y='Count', kind='bar', color= "cadetblue")
#setting x and y labels 
plt.xlabel('Team')
plt.ylabel('Championships won')
#title
plt.title('Championships won by team')


Is Age an importan factor? 

The average age in the winer teams is narrowed between 26.6 to 31.7

In [ ]:
#df Winners Teams and age 
df_team_age = df_won_champ[["Team" , "Age"]]

df_team_age

In [ ]:
df_team_age.min()
df_team_age.max()


In [ ]:
#Plotting winners teams vs age 
df_team_age = df_won_champ[["Team" , "Age"]]

df_team_age.plot(x='Team', y='Age', kind='bar', color= "deeppink")
#laxis imits
plt.ylim(20, 40) 
#setting x and y labels  
plt.xlabel('Team')
plt.ylabel('Age')
#title 
plt.title('Team vs Age')



Understanding if age correlates directly with wins

In [ ]:
#Team vs age 
df_age_wins = df_won_champ[["Age" , "WINS"]]

df_age_wins.plot.scatter(x='Age', y='WINS', color= "orange")
#axis limits
plt.ylim(30, 100) 
#setting x and y labels  
plt.xlabel('Age')
plt.ylabel('Wins')
#title 
plt.title('Age vs Wins')

In [ ]:
df_age_wins = df_won_champ[["OFFENSIVE REBOUND PERCENTAGE" , "MARGIN OF VICTORY"]]

df_age_wins.plot.scatter(x='OFFENSIVE REBOUND PERCENTAGE', y="MARGIN OF VICTORY", color= "orange")
#axis limits
#plt.ylim(30, 100) 
plt.xlabel('Offensive Rebound %')
plt.ylabel('Margin of Victory')
#title 
plt.title('Margin of Victory vs Offensive Rebound %')

### Using OneHotEncoder for Teams which is a categorical data. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
encodeded_teams = encoder.fit_transform(df_renamed[['Team']])

In [ ]:
encodeded_teams_df = pd.DataFrame(encodeded_teams, columns=encoder.get_feature_names_out(["Team"]))

In [ ]:
encodeded_teams_df.head()

In [ ]:
df_renamed.drop(df_renamed.columns[2], axis=1, inplace=True)

df_concat = pd.concat([df_renamed,encodeded_teams_df], axis=1)

df_concat

## Part 3 Creating the Model 

Creating and comparing two models on this data: a Logistic Regression, and a Random Forests Classifier.

Split the Data into Training and Testing Sets

In [ ]:
# Create the labels set `y` and features DataFrame `X`
y = df_renamed['Championship']
X = df_renamed.drop(columns=['Championship'])
display(y.head())
display(X.head())


In [ ]:
# Check the balance of the labels variable (`y`) by using the `value_counts` function.
y.value_counts()

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [ ]:
num_rows = y_train.shape[0]
num_rows

### Scale the Features
Use the `StandardScaler` to scale the features data. Only `X_train` and `X_test` DataFrames should be scaled.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create the StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Fit the Standard Scaler with the training data
scaler.fit(X_train)

# Fit the Standard Scaler with the test data
scaler.fit(X_test)

In [ ]:
# Scale the training data
X_train_scaled = scaler.transform(X_train)

# Scale the test data
X_test_scaled = scaler.transform(X_test)

### Creation and Fitting Logistic Regression Model

Creating a Logistic Regression model, fitting it to the training data, making predictions with the testing data, and printing the model's accuracy score.

In [ ]:
# Make and save testing predictions with the saved logistic regression model using the test data
logistic_regression_model = LogisticRegression(random_state=1)
logistic_regression_model.fit(X_train_scaled, y_train)
# Review the predictions
y_pred = logistic_regression_model.predict(X_test_scaled)
y_pred

In [ ]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled, y_train):.3f}")
print(f"Testing Data Score: {logistic_regression_model.score(X_test_scaled, y_test):.3f}")

# Generate predictions from the model we just fit
testing_predictions_lr = logistic_regression_model.predict(X_test_scaled)

# Display the accuracy score for the testing dataset
print('Logistic Regression Model Accuracy Score: %.3f' % accuracy_score(y_test, testing_predictions_lr) )

### Using Random Forest Classifier Model

Using Random Forest Classifier model to fit the training data, make predictions with the testing data, and print the model's accuracy score.

In [ ]:
# Make and save testing predictions with the saved logistic regression model using the test data
random_forest_model = RandomForestClassifier(random_state=1)
random_forest_model.fit(X_train_scaled, y_train)

# Review the predictions
y_pred = random_forest_model.predict(X_train_scaled)
y_pred

In [ ]:
# Calculate the accuracy score by evaluating `y_test` vs. `testing_predictions`.
print(f"Testing Data Score: {random_forest_model.score(X_test_scaled, y_test):.3f}")

# Part 4 Data Preparation for Modeling

Next, we will prepare the data for modeling. We can choose a target variable, such as "Championship" or "Playoffs," and perform the following steps:

In [ ]:
# Identify unique values in the 'Championship' column
print(df_concat['Championship'].unique())

# Replace 'Null' or other non-numeric values with NaN
df_concat['Championship'].replace('Null', np.nan, inplace=True)

# Assuming 'Null' means no championship, we replace with 0
df_concat['Championship'].fillna(0, inplace=True)

# Convert the column to integers
df_concat['Championship'] = df_concat['Championship'].astype(int)

# Verify the conversion
print(df_concat['Championship'].unique())

# Check for missing values
print(df_concat.isnull().sum())

# Summary statistics
print(df_concat.describe())

# Select only numeric columns
numeric_df = df_concat.select_dtypes(include=[np.number])

# Feature selection
features = ["WINS", "LOSSES", "PYTHAGOREAN WINS", "PYTHAGOREAN LOSSES", 
            "MARGIN OF VICTORY", "STRENGTH OF SCHEDULE", "SIMPLE RATING SYSTEM", 
            "OFFENSIVE RATING", "DEFENSIVE RATING", "PACE FACTOR", 
            "FREE THROW ATTEMPT RATE", "3-POINT ATTEMPT RATE", "TRUE SHOOTING PERCENTAGE", 
            "EFFECTIVE FIELD GOAL PERCENTAGE", "TURNOVER PERCENTAGE", 
            "OFFENSIVE REBOUND PERCENTAGE", "FREE THROWS PER FIELD GOAL ATTEMPT", 
            "OPPONENT EFFECTIVE FIELD GOAL PERCENTAGE", "OPPONENT TURNOVER PERCENTAGE", 
            "DEFENSIVE REBOUND PERCENTAGE", "OPPONENT FREE THROWS PER FIELD GOAL ATTEMPT"]

X = numeric_df[features]
y = df_concat['Championship']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

# Visualize confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_log_reg)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

The logistic regression model achieves an accuracy of approximately 98.40% on the test set, with a confusion matrix showing 183 true negatives (no championship) and 2 true positives (championship). However, the classification report indicates some challenges in classifying the positive class (championship), with a lower recall and F1-score due to only 2 true positives out of 5 total positive instances in the test set.

Given this information, it's important to note that the class imbalance in the target variable (championship) might affect the model's performance, especially in terms of correctly identifying instances of championship wins.

In [ ]:
# Identify unique values in the 'Playoffs' column
print(df_concat['Playoffs'].unique())

# Replace 'Null' or other non-numeric values with NaN
df_concat['Playoffs'].replace('Null', np.nan, inplace=True)

# Assuming 'Null' means no Playoffs, we replace with 0
df_concat['Playoffs'].fillna(0, inplace=True)

# Convert the column to integers
df_concat['Playoffs'] = df_concat['Playoffs'].astype(int)

# Verify the conversion
print(df_concat['Playoffs'].unique())

# Check for missing values
print(df_concat.isnull().sum())

# Summary statistics
print(df_concat.describe())

# Select only numeric columns
numeric_df_1 = df_concat.select_dtypes(include=[np.number])

# Feature selection
features_1 = ["WINS", "LOSSES", "PYTHAGOREAN WINS", "PYTHAGOREAN LOSSES", 
            "MARGIN OF VICTORY", "STRENGTH OF SCHEDULE", "SIMPLE RATING SYSTEM", 
            "OFFENSIVE RATING", "DEFENSIVE RATING", "PACE FACTOR", 
            "FREE THROW ATTEMPT RATE", "3-POINT ATTEMPT RATE", "TRUE SHOOTING PERCENTAGE", 
            "EFFECTIVE FIELD GOAL PERCENTAGE", "TURNOVER PERCENTAGE", 
            "OFFENSIVE REBOUND PERCENTAGE", "FREE THROWS PER FIELD GOAL ATTEMPT", 
            "OPPONENT EFFECTIVE FIELD GOAL PERCENTAGE", "OPPONENT TURNOVER PERCENTAGE", 
            "DEFENSIVE REBOUND PERCENTAGE", "OPPONENT FREE THROWS PER FIELD GOAL ATTEMPT"]

X = numeric_df_1[features]
y = numeric_df_1['Playoffs']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

# Visualize confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_log_reg)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

The logistic regression model achieves an accuracy of approximately 96.28% on the test set. The confusion matrix shows that out of 188 samples, 86 were correctly predicted as not making the playoffs (true negatives), 95 were correctly predicted as making the playoffs (true positives), 6 were incorrectly predicted as making the playoffs when they didn't (false positives), and 1 was incorrectly predicted as not making the playoffs when it did (false negative). The classification report provides precision, recall, F1-score, and support for both classes (0 and 1).

Overall, the logistic regression model appears to perform well on the given dataset for predicting playoff appearances based on the selected features.

In [ ]:
from imblearn.over_sampling import SMOTE

# Replace 'Null' or other non-numeric values with NaN
df_concat['Playoffs'].replace('Null', np.nan, inplace=True)

# Fill NaN values with 0
df_concat['Playoffs'].fillna(0, inplace=True)

# Convert the column to integers
df_concat['Playoffs'] = df_concat['Playoffs'].astype(int)

# Select only numeric columns
numeric_df_2 = df_concat.select_dtypes(include=[np.number])

# Feature selection
features_2 = ["WINS", "LOSSES", "PYTHAGOREAN WINS", "PYTHAGOREAN LOSSES", 
            "MARGIN OF VICTORY", "STRENGTH OF SCHEDULE", "SIMPLE RATING SYSTEM", 
            "OFFENSIVE RATING", "DEFENSIVE RATING", "PACE FACTOR", 
            "FREE THROW ATTEMPT RATE", "3-POINT ATTEMPT RATE", "TRUE SHOOTING PERCENTAGE", 
            "EFFECTIVE FIELD GOAL PERCENTAGE", "TURNOVER PERCENTAGE", "OFFENSIVE REBOUND PERCENTAGE", 
            "FREE THROWS PER FIELD GOAL ATTEMPT", "OPPONENT EFFECTIVE FIELD GOAL PERCENTAGE", 
            "OPPONENT TURNOVER PERCENTAGE", "DEFENSIVE REBOUND PERCENTAGE", 
            "OPPONENT FREE THROWS PER FIELD GOAL ATTEMPT"]

X = numeric_df_2[features]
y = numeric_df_2['Playoffs']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_smote, y_train_smote)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Random Forest Classifier Accuracy:", accuracy)
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

The Random Forest Classifier achieves an accuracy of approximately 94.15% on the test set. The confusion matrix shows that out of 188 samples, 83 were correctly predicted as not making the playoffs (true negatives), 94 were correctly predicted as making the playoffs (true positives), 9 were incorrectly predicted as making the playoffs when they didn't (false positives), and 2 were incorrectly predicted as not making the playoffs when they did (false negatives). The classification report provides precision, recall, F1-score, and support for both classes (0 and 1).

The Random Forest Classifier performs slightly worse than the logistic regression model in terms of accuracy but still achieves strong results in predicting playoff appearances based on the selected features.

The Random Forest Classifier model achieves an accuracy of approximately 93.62% on the test set. Both models perform well, with the Logistic Regression model slightly outperforming the Random Forest Classifier in terms of accuracy and precision for predicting both classes. However, the Random Forest Classifier exhibits slightly higher recall for class 1 (making playoffs), indicating its ability to better identify instances of this class.

In [ ]:
# Identify unique values in the 'NET RATING' column
print(df_concat['NET RATING'].unique())

# Replace 'Null' or other non-numeric values with NaN
df_concat['NET RATING'].replace('Null', np.nan, inplace=True)

# Assuming 'Null' means no NET RATING, we replace with 0
df_concat['NET RATING'].fillna(0, inplace=True)

# Convert the column to integers
df_concat['NET RATING'] = df_concat['NET RATING'].astype(int)

# Verify the conversion
print(df_concat['NET RATING'].unique())

# Check for missing values
print(df_concat.isnull().sum())

# Summary statistics
print(df_concat.describe())

# Select only numeric columns
numeric_df_3 = df_concat.select_dtypes(include=[np.number])

# Feature selection
features_3 = ["WINS", "LOSSES", "PYTHAGOREAN WINS", "PYTHAGOREAN LOSSES", 
            "MARGIN OF VICTORY", "STRENGTH OF SCHEDULE", "SIMPLE RATING SYSTEM", 
            "OFFENSIVE RATING", "DEFENSIVE RATING", "PACE FACTOR", 
            "FREE THROW ATTEMPT RATE", "3-POINT ATTEMPT RATE", "TRUE SHOOTING PERCENTAGE", 
            "EFFECTIVE FIELD GOAL PERCENTAGE", "TURNOVER PERCENTAGE", 
            "OFFENSIVE REBOUND PERCENTAGE", "FREE THROWS PER FIELD GOAL ATTEMPT", 
            "OPPONENT EFFECTIVE FIELD GOAL PERCENTAGE", "OPPONENT TURNOVER PERCENTAGE", 
            "DEFENSIVE REBOUND PERCENTAGE", "OPPONENT FREE THROWS PER FIELD GOAL ATTEMPT"]

X = numeric_df_3[features]
y = numeric_df_3['NET RATING']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

# Visualize confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_log_reg)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()